# Closed-Loop Evaluation
In this notebook you are going to evaluate Urban Driver to control the SDV with a protocol named *closed-loop* evaluation.

**Note: this notebook assumes you've already run the [training notebook](./train.ipynb) and stored your model successfully (or that you have stored a pre-trained one).**

**Note: for a detailed explanation of what closed-loop evaluation (CLE) is, please refer to our [planning notebook](../planning/closed_loop_test.ipynb)**

### Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from prettytable import PrettyTable

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset

from l5kit.dataset import EgoDataset, EgoDatasetVectorized
from l5kit.vectorization.vectorizer_builder import build_vectorizer
from l5kit.rasterization.rasterizer_builder import build_rasterizer

from l5kit.simulation.dataset import SimulationConfig
from l5kit.simulation.unroll import ClosedLoopSimulator
from l5kit.cle.closed_loop_evaluator import ClosedLoopEvaluator, EvaluationPlan
from l5kit.cle.metrics import (CollisionFrontMetric, CollisionRearMetric, CollisionSideMetric,
                               DisplacementErrorL2Metric, DistanceToRefTrajectoryMetric)
from l5kit.cle.validators import RangeValidator, ValidationCountingAggregator

from l5kit.visualization.visualizer.zarr_utils import simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.data import MapAPI

from collections import defaultdict
import os

from custom_closed_loop_simulator import CustomClosedLoopSimulator

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params ...).

In [3]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "./tmp/l5kit_data"
dm = LocalDataManager(None)

# get config
agents_cfg = load_config_data("./tmp/agents_model/agents_config.yaml")
ego_cfg = load_config_data("./tmp/ego_model/ego_config.yaml")

## Load the model

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

agents_model_path = "./tmp/agents_model/agents_model.pt"
agents_model = torch.load(agents_model_path).to(device)
agents_model = agents_model.eval()

ego_model_path = "./tmp/ego_model/ego_model.pt"
ego_model = torch.load(ego_model_path).to(device)
ego_model = ego_model.eval()

torch.set_grad_enabled(False)

C:\Users\thver\AppData\Roaming\Python\Python38\site-packages\torch\serialization.py:602: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


## Load the evaluation data
Differently from training and open loop evaluation, this setting is intrinsically sequential. As such, we won't be using any of PyTorch's parallelisation functionalities.

In [5]:
# ===== INIT DATASET

eval_agents_cfg = agents_cfg["val_data_loader"]
eval_agents_zarr = ChunkedDataset(dm.require(eval_agents_cfg["key"])).open()
rasterizer = build_rasterizer(agents_cfg, dm)
agents_dataset = EgoDataset(agents_cfg, eval_agents_zarr, rasterizer)
print(agents_dataset)

eval_ego_cfg = ego_cfg["val_data_loader"]
eval_ego_zarr = ChunkedDataset(dm.require(eval_ego_cfg["key"])).open()
vectorizer = build_vectorizer(ego_cfg, dm)
ego_dataset = EgoDatasetVectorized(ego_cfg, eval_ego_zarr, vectorizer)
print(ego_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|    100     |   24838    |  1893736   |     316008    |       0.69      |        248.38        |        76.24         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Nu

## Define some simulation properties
We define here some common simulation properties such as the length of the simulation and how many scene to simulate.

**NOTE: these properties have a significant impact on the execution time. We suggest you to increase them only if your setup includes a GPU.**

In [6]:
num_scenes_to_unroll = 10
num_simulation_steps = 50

# Closed-loop simulation

We define a closed-loop simulation that drives the SDV for `num_simulation_steps` steps while using the log-replayed agents.

Then, we unroll the selected scenes.
The simulation output contains all the information related to the scene, including the annotated and simulated positions, states, and trajectories of the SDV and the agents.  
If you want to know more about what the simulation output contains, please refer to the source code of the class `SimulationOutput`.

In [7]:
# ==== DEFINE CLOSED-LOOP SIMULATION
sim_cfg = SimulationConfig(use_ego_gt=False, use_agents_gt=False, disable_new_agents=True,
                           distance_th_far=500, distance_th_close=50, num_simulation_steps=num_simulation_steps,
                           start_frame_index=0, show_info=True)

sim_loop = CustomClosedLoopSimulator(sim_cfg, agents_dataset, ego_dataset, device, model_ego=ego_model, model_agents=agents_model)

In [8]:
# ==== UNROLL
scenes_to_unroll = list(range(0, len(eval_ego_zarr.scenes), len(eval_ego_zarr.scenes)//num_scenes_to_unroll))
sim_outs = sim_loop.unroll(scenes_to_unroll)

NameError: name 'eval_zarr' is not defined

# Closed-loop metrics

**Note: for a detailed explanation of CLE metrics, please refer again to our [planning notebook](../planning/closed_loop_test.ipynb)**

In [ ]:
metrics = [DisplacementErrorL2Metric(),
           DistanceToRefTrajectoryMetric(),
           CollisionFrontMetric(),
           CollisionRearMetric(),
           CollisionSideMetric()]

validators = [RangeValidator("displacement_error_l2", DisplacementErrorL2Metric, max_value=30),
              RangeValidator("distance_ref_trajectory", DistanceToRefTrajectoryMetric, max_value=4),
              RangeValidator("collision_front", CollisionFrontMetric, max_value=0),
              RangeValidator("collision_rear", CollisionRearMetric, max_value=0),
              RangeValidator("collision_side", CollisionSideMetric, max_value=0)]

intervention_validators = ["displacement_error_l2",
                           "distance_ref_trajectory",
                           "collision_front",
                           "collision_rear",
                           "collision_side"]

cle_evaluator = ClosedLoopEvaluator(EvaluationPlan(metrics=metrics,
                                                   validators=validators,
                                                   composite_metrics=[],
                                                   intervention_validators=intervention_validators))

# Quantitative evaluation

We can now compute the metric evaluation, collect the results and aggregate them.

In [ ]:
cle_evaluator.evaluate(sim_outs)
validation_results = cle_evaluator.validation_results()
agg = ValidationCountingAggregator().aggregate(validation_results)
cle_evaluator.reset()

## Reporting errors from the closed-loop

We can now report the metrics and plot them.

In [ ]:
fields = ["metric", "value"]
table = PrettyTable(field_names=fields)

values = []
names = []

for metric_name in agg:
    table.add_row([metric_name, agg[metric_name].item()])
    values.append(agg[metric_name].item())
    names.append(metric_name)

print(table)

plt.bar(np.arange(len(names)), values)
plt.xticks(np.arange(len(names)), names, rotation=60, ha='right')
plt.show()

# Qualitative evaluation

## Visualise the closed-loop

We can visualise the scenes we have obtained previously. 

**The policy is now in full control of the SDV as this moves through the annotated scene.**

In [ ]:
output_notebook()
mapAPI = MapAPI.from_cfg(dm, cfg)
for sim_out in sim_outs: # for each scene
    vis_in = simulation_out_to_visualizer_scene(sim_out, mapAPI)
    show(visualize(sim_out.scene_id, vis_in))